In [1]:
!git clone https://github.com/timothycao/agnews-classifier.git
%cd agnews-classifier

Cloning into 'agnews-classifier'...
remote: Enumerating objects: 37, done.
remote: Counting objects: 100% (37/37), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 37 (delta 17), reused 28 (delta 9), pack-reused 0 (from 0)
Receiving objects: 100% (37/37), 10.68 KiB | 3.56 MiB/s, done.
Resolving deltas: 100% (17/17), done.
/kaggle/working/agnews-classifier


In [51]:
# Model

from peft import LoraConfig
from model import create_lora_model

lora_config = LoraConfig(
    r=3,
    lora_alpha=9,
    lora_dropout=0.1,
    bias='none',
    target_modules=['query', 'key', 'value', 'attention.output.dense'],
    task_type='SEQ_CLS',
    use_rslora = False # regular LoRA: lora_alpha/r, rs-LoRA: lora_alpha/sqrt(r) -> normalization technique
)

model = create_lora_model(lora_config)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 814,852 || all params: 125,463,560 || trainable%: 0.6495


In [52]:
model

PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): RobertaForSequenceClassification(
      (roberta): RobertaModel(
        (embeddings): RobertaEmbeddings(
          (word_embeddings): Embedding(50265, 768, padding_idx=1)
          (position_embeddings): Embedding(514, 768, padding_idx=1)
          (token_type_embeddings): Embedding(1, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): RobertaEncoder(
          (layer): ModuleList(
            (0-11): 12 x RobertaLayer(
              (attention): RobertaAttention(
                (self): RobertaSdpaSelfAttention(
                  (query): lora.Linear(
                    (base_layer): Linear(in_features=768, out_features=768, bias=True)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.1, inplace=False)
                    )
                    (lora_A): Mod

In [53]:
# Training

from transformers import TrainingArguments
from train import main as train

training_args = TrainingArguments(
    # Core training configs
    max_steps=2000,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    optim='adamw_torch',
    learning_rate=5e-5,
    lr_scheduler_type='linear',
    
    # Logging, evaluation, and checkpointing
    output_dir='/kaggle/working/saved_models',
    logging_strategy='steps',
    logging_steps=100,
    eval_strategy='steps',
    eval_steps=100,
    save_strategy='steps',
    save_steps=100,

    # Miscellaneous
    report_to='none',
    dataloader_num_workers=4,
    gradient_checkpointing=False,
    gradient_checkpointing_kwargs={'use_reentrant':True}
)

train(model, training_args)

Starting training...


Step,Training Loss,Validation Loss,Accuracy
100,1.359200,1.289505,0.856250
200,0.781400,0.367761,0.879687
300,0.376100,0.353587,0.885938
400,0.292100,0.349718,0.893750
500,0.313800,0.329674,0.901563
600,0.302400,0.328969,0.900000
700,0.301800,0.327958,0.901563
800,0.319300,0.316855,0.904687
900,0.296500,0.311777,0.904687
1000,0.287200,0.307831,0.906250


In [55]:
# Inference

from inference import main as inference

data_path = '/kaggle/input/deep-learning-spring-2025-project-2/test_unlabelled.pkl'
checkpoint = '/kaggle/working/saved_models/checkpoint-2000'
output_dir = '/kaggle/working/saved_predictions'

inference(data_path, checkpoint, output_dir)

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Running inference...


100%|██████████| 250/250 [00:59<00:00,  4.18it/s]

Predictions saved to /kaggle/working/saved_predictions/predictions_checkpoint-2000.csv


In [54]:
import pandas as pd

csv_path = '/kaggle/working/saved_models/checkpoint-2000/processed_log_history.csv'

df = pd.read_csv(csv_path)
df

,Step,Train Loss,Test Loss,Train Acc,Test Acc,Learning Rate,Epochs,Loss Spread,Loss Ratio,Acc Spread,Acc Ratio
0,100,1.3592,1.289505,0.389375,0.856250,0.000048,0.013405,0.069695,1.054048,-0.466875,0.454745
1,200,0.7814,0.367761,0.825000,0.879687,0.000045,0.026810,0.413639,2.124748,-0.054688,0.937833
2,300,0.3761,0.353587,0.874107,0.885938,0.000043,0.040214,0.022513,1.063671,-0.011830,0.986647
3,400,0.2921,0.349718,0.901339,0.893750,0.000040,0.053619,-0.057618,0.835245,0.007589,1.008492
4,500,0.3138,0.329674,0.893304,0.901563,0.000037,0.067024,-0.015874,0.951849,-0.008259,0.990839
5,600,0.3024,0.328969,0.898214,0.900000,0.000035,0.080429,-0.026569,0.919234,-0.001786,0.998016
6,700,0.3018,0.327958,0.903125,0.901563,0.000033,0.093834,-0.026158,0.920241,0.001562,1.001733
7,800,0.3193,0.316855,0.897768,0.904687,0.000030,0.107239,0.002445,1.007717,-0.006920,0.992351
8,900,0.2965,0.311777,0.899554,0.904687,0.000028,0.120643,-0.015277,0.950999,-0.005134,0.994325
9,1000,0.2872,0.307831,0.905357,0.906250,0.000025,0.134048,-0.020631,0.932978,-0.000893,0.999015
